In [2]:
import os
import shutil
import glob
import numpy as np
import pandas as pd



In [ ]:
from torchvision.datasets import DatasetFolder
from PIL import Image
from torchvision import transforms
import torch
import pandas as pd



class GlucoseDataset(DatasetFolder):
    def __init__(self, img_dir, dataframe, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]
        glucose_level = self.dataframe.iloc[idx, 1]
        image = Image.open(img_path).convert('RGB')  # Convert grayscale images to RGB
        if self.transform:
            image = self.transform(image)
        return image, glucose_level

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

In [ ]:

data_dir = "glucose-prediction/RP_Images/"


train_df = pd.read_csv('/glucose-prediction/Mappings/train_mappings.csv')
val_df = pd.read_csv('/glucose-prediction/Mappings/val_mappings.csv')


train_dataset = GlucoseDataset(data_dir + 'train', train_df, transform=data_transforms['train'])
val_dataset = GlucoseDataset(data_dir + 'val', val_df, transform=data_transforms['val'])






In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.optim.lr_scheduler import StepLR

from torchvision.models import resnet18
from torchvision.models.resnet import ResNet18_Weights

model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

# Loss functions
def rmse(y_true, y_pred):
    return torch.sqrt(torch.mean((y_pred - y_true) ** 2))

def mape(y_true, y_pred):
    return torch.mean(torch.abs((y_true - y_pred) / (y_true + 1e-8)))

def combined_loss(y_true, y_pred):
    return rmse(y_true, y_pred) + mape(y_true, y_pred)

epochs = 25

# possible learning rates and batch sizes
learning_rates = [0.1, 0.01, 0.001]
batch_sizes = [16, 32, 64]


best_val_loss = float('inf')
best_lr = None
best_batch_size = None

print(f"Planning to run for {epochs} epochs with learning rates: {learning_rates} and batch sizes: {batch_sizes}")

for lr in learning_rates:
    for batch_size in batch_sizes:
        print(f"\nStarting training with learning rate: {lr} and batch size: {batch_size}")

        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
        val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

        optimizer = torch.optim.SGD(model.parameters(), lr=lr)
        step_size = 10
        gamma = 0.1
        scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

        # Train the model
        for epoch in range(epochs):
            print(f"\nStarting epoch {epoch + 1}/{epochs}")

            model.train()
            total_train_loss = 0.0

            for images, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(images)
                loss = combined_loss(labels, outputs)
                total_train_loss += loss.item()
                loss.backward()
                optimizer.step()

            scheduler.step()

            avg_train_loss = total_train_loss / len(train_loader)
            print(f"Epoch {epoch + 1}, Training Loss: {avg_train_loss:.4f}")

            # Validate the model
            model.eval()
            total_val_loss = 0.0

            with torch.no_grad():
                for images, labels in val_loader:
                    outputs = model(images)
                    total_val_loss += combined_loss(labels, outputs).item()

            avg_val_loss = total_val_loss / len(val_loader)
            print(f"Epoch {epoch + 1}, Validation Loss: {avg_val_loss:.4f}")

            # Check if the current model is the best so far
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                best_lr = lr
                best_batch_size = batch_size
                torch.save(model.state_dict(), 'best_model_weights.pth')

print("\nBest learning rate:", best_lr)
print("Best batch size:", best_batch_size)



Planning to run for 25 epochs with learning rates: [0.1, 0.01, 0.001] and batch sizes: [16, 32, 64]

Starting training with learning rate: 0.1 and batch size: 16

Starting epoch 1/25
Epoch 1, Training Loss: 28.3955
Epoch 1, Validation Loss: 6783.0180

Starting epoch 2/25
Epoch 2, Training Loss: 24.2807
Epoch 2, Validation Loss: 23.0766

Starting epoch 3/25
Epoch 3, Training Loss: 23.9707
Epoch 3, Validation Loss: 24.7405

Starting epoch 4/25
Epoch 4, Training Loss: 23.1480
Epoch 4, Validation Loss: 20.1613

Starting epoch 5/25
Epoch 5, Training Loss: 22.8252
Epoch 5, Validation Loss: 21.4883

Starting epoch 6/25
Epoch 6, Training Loss: 23.0028
Epoch 6, Validation Loss: 20.6856

Starting epoch 7/25
Epoch 7, Training Loss: 23.0307
Epoch 7, Validation Loss: 23.8454

Starting epoch 8/25
Epoch 8, Training Loss: 22.5189
Epoch 8, Validation Loss: 23.1450

Starting epoch 9/25
Epoch 9, Training Loss: 22.0112
Epoch 9, Validation Loss: 20.3556

Starting epoch 10/25
Epoch 10, Training Loss: 22.076